In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime

In [ ]:
# =========================================================
# CONFIG
# =========================================================
base_dir = Path(r" ")
input_root = base_dir / "Step2"
output_root = base_dir / "Step3"
os.makedirs(output_root, exist_ok=True)

# intervention week (use week containing 2020-03-11)
intervention_date = pd.to_datetime("2020-03-11")
global_start = pd.to_datetime("2010-01-04")
global_end   = pd.to_datetime("2021-12-26")
weekly_index = pd.date_range(start=global_start, end=global_end, freq="W-MON")

def week_start(ts):
    return ts.to_period("W-SUN").start_time if pd.notna(ts) else pd.NaT

In [3]:
# =========================================================
# CATEGORY MAPPING
# =========================================================
category_map = {
    # Politicians & world leaders
    "Politicians and World Leaders": [
        "Abdelfattah Elsisi","Alberto Fernández","Barack Obama","Benjamin Netanyahu","Boris Johnson",
        "Emmanuel Macron","Imran Khan","Ivan Duque","Jair Bolsonaro","Joko Widodo","Justin Trudeau",
        "Kaguta Museveni","King Salman","Lopez Obrador","Mohammed AlMaktoum","Moon Jaein",
        "Muhammadu Buhari","Nana AkufoAddo","Narendra Modi","Nicolas Maduro","Paul Kagame","Queen Rania","Sebastian Pinera","Tayyip Erdogan"
    ],

    # Business / entrepreneurs
    "Business Leaders and Entrepreneurs": [
        "Bill Gates","Elon Musk","Eric Yuan","Jack Dorsey","Joe Gebbia","John Collison","Ken Fisher",
        "Marc Benioff","Melinda Gates","Michael Dell","Micky Arison","Mike Bloomberg","Mike Brookes",
        "Orlando Bravo","Patrick Collison","Ralph Lauren","Ricardo Salinas","Samuel Bankman",
        "Tilman Fertitta","Tim Sweeney","Tobi Lutke","Vinod Khosla"
    ],

    # Entertainment umbrella: actors, athletes, comedians, models
    "Entertainers and Celebrities": [
        # singers, actors, athletes, comedians, reality stars
        "Adele Adkins","Alecia Beth","Alicia Keys","Armando Perez","Britney Spears","Bruno Mars",
        "Chris Brown","Demi Lovato","Drake Graham","Harry Styles","Jennifer Lopez","Justin Bieber",
        "Justin Timberlake","Kanye West","Katy Perry","Lady Gaga","Liam Payne","Lil Wayne",
        "Louis Tomlinson","Miley Cyrus","Niall Horan","Nicki Minaj","Robyn Rihanna","Shakira Ripoll",
        "Shawn Mendes","Taylor Swift","Wiz Khalifa","Zayn Malik",
        "Akshay Kumar","Amitabh Bachchan","Deepika Padukone","Emma Watson","Hrithik Roshan",
        "Priyanka Chopra","Salman Khan","Selena Gomez","ShahRukh Khan",
        "Andres Iniesta","Cristiano Ronaldo","LeBron James","Mesut Ozil","Neymar Junior",
        "Ricardo Kaka","Sachin Tendulkar","Virat Kohli",
        "Conan Obrien","Jimmy Fallon","Kevin Hart","Oprah Winfrey","Patrick Harris","Whindersson Nunes",
        "Kendal Jenner","Khloe Kardashian","Kim Kardashian","Kourtney Kardashian","Kylie Jenner"
    ]
}

# reverse map for lookup
person_to_cat = {p: c for c, plist in category_map.items() for p in plist}

In [ ]:
# =========================================================
# BUILD BALANCED PANEL (FIXED VERSION)
# =========================================================

def normalize_name(name: str) -> str:
    """Normalize a name by removing spaces, underscores, and making lowercase."""
    return name.replace(" ", "").replace("_", "").lower()

# Build normalized lookup dictionary from category_map
person_to_cat = {
    normalize_name(person): category
    for category, people in category_map.items()
    for person in people
}

panel_rows = []

for folder in input_root.glob("*"):
    for f in folder.glob("*.xlsx"):
        # Load file
        df = pd.read_excel(f)
        df["week"] = pd.to_datetime(df["week"], errors="coerce").dt.normalize()
        
        # Ensure every person has all weeks (balanced panel)
        reindexed = pd.DataFrame({"week": weekly_index}).merge(df, on="week", how="left")
        reindexed["tweet_count"] = reindexed["tweet_count"].fillna(0).astype(int)
        
        # Extract and normalize person name
        person = f.stem.replace("_Weekly_Behavioral_Summary", "")
        person_norm = normalize_name(person)
        
        # Assign category using normalized lookup
        category = person_to_cat.get(person_norm, "Unknown")
        
        # Add columns
        reindexed["person"] = person
        reindexed["category"] = category
        
        panel_rows.append(reindexed)

# Combine all people into one panel DataFrame
panel = pd.concat(panel_rows, ignore_index=True)

# Add time-related variables
panel["time"] = ((panel["week"] - global_start) / pd.Timedelta(weeks=1)).astype(int)
panel["intervention"] = (panel["week"] >= week_start(intervention_date)).astype(int)
panel["time_after"] = panel["time"] * panel["intervention"]
panel["person_str"] = panel["person"].astype(str)

# Save panel
output_path = output_root / "All_People_Weekly_Balanced_Panel.csv"
panel.to_csv(output_path, index=False)

print(f"✅ Panel ready: {panel.shape}")
print(panel["category"].value_counts(dropna=False).head(10))

In [ ]:
# =========================================================
# RUN ITS MODELS (BY CATEGORY)
# =========================================================
def fit_model(df, formula, family=None):
    if family:
        model = smf.glm(formula, data=df, family=family).fit(
            cov_type="cluster", cov_kwds={"groups": df["person"]}
        )
    else:
        model = smf.ols(formula, data=df).fit(
            cov_type="cluster", cov_kwds={"groups": df["person"]}
        )
    return model

outcomes = ["tweet_count","avg_sentiment","avg_length",
            "prop_tweet","prop_retweet","prop_reply","prop_quote"]

all_results = []

for cat, df_cat in panel.groupby("category"):
    print(f"\n📊 Category: {cat} (n={df_cat.person.nunique()} people)")
    for var in outcomes:
        if var not in df_cat.columns: continue
        df_sub = df_cat.dropna(subset=[var])
        family = sm.families.Poisson() if var=="tweet_count" else None
        model = fit_model(df_sub, f"{var} ~ time + intervention + time_after + C(person_str)", family)
        resfile = output_root / f"ITS_{var}_{cat.replace(' ','_')}_summary.txt"
        with open(resfile,"w",encoding="utf-8") as f: f.write(model.summary().as_text())
        all_results.append((cat,var,model))

In [ ]:
# =========================================================
# EXTRACT COEFFICIENTS & SIGNIFICANCE
# =========================================================
summary_data = []
for cat,var,model in all_results:
    for term in ["intervention","time","time_after"]:
        if term in model.params:
            summary_data.append({
                "category":cat,
                "variable":var,
                "term":term,
                "coef":model.params[term],
                "std_err":model.bse[term],
                "p_value":model.pvalues[term],
                "z":model.tvalues[term]
            })
summary_df = pd.DataFrame(summary_data)

# significance stars
def stars(p):
    return "***" if p<0.001 else "**" if p<0.01 else "*" if p<0.05 else ""

summary_df["sig"] = summary_df["p_value"].apply(stars)
summary_df["coef_sig"] = summary_df["coef"].round(4).astype(str)+summary_df["sig"]

summary_path = output_root / "ITS_summary_by_category.csv"
summary_df.to_csv(summary_path,index=False)
print(f"✅ Saved category-level summary to {summary_path}")

In [ ]:
# =========================================================
# PERSON-LEVEL META REGRESSION (robust)
# =========================================================
import numpy as np
import statsmodels.formula.api as smf

# outcomes to meta-analyze (same names used earlier)
outcomes = ["tweet_count","avg_sentiment","avg_length",
            "prop_tweet","prop_retweet","prop_reply","prop_quote"]

# Ensure person->category mapping normalized (if you used normalized mapping earlier)
# person_to_cat: keys are exact person strings as stored in panel["person"]
# followers: dict mapping display name -> follower count (you already have this)

# Build raw meta rows: for each person & outcome, fit simple ITS and keep coefficients
meta_rows = []
min_rows_required = 15  # you used 15 earlier

for var in outcomes:
    print(f"\n🔍 Building person-level ITS stats for: {var}")
    for person, d in panel.groupby("person"):
        # require enough non-NaN data points for this person for this variable
        if d[var].notna().sum() < min_rows_required:
            continue
        # dropna on covariates (time/intervention) too
        dsub = d.dropna(subset=[var, "time", "intervention", "time_after"])
        if dsub.shape[0] < min_rows_required:
            continue
        try:
            m = smf.ols(f"{var} ~ time + intervention + time_after", data=dsub).fit()
            meta_rows.append({
                "person": person,
                "variable": var,
                "n_obs": dsub.shape[0],
                "intervention_coef": m.params.get("intervention", np.nan),
                "time_coef": m.params.get("time", np.nan),
                "time_after_coef": m.params.get("time_after", np.nan),
                "intervention_se": m.bse.get("intervention", np.nan),
                "time_se": m.bse.get("time", np.nan),
                "time_after_se": m.bse.get("time_after", np.nan),
                "intervention_p": m.pvalues.get("intervention", np.nan),
                "time_p": m.pvalues.get("time", np.nan),
                "time_after_p": m.pvalues.get("time_after", np.nan),
            })
        except Exception as e:
            # log optional: print(person, var, "err", e)
            pass

meta_df = pd.DataFrame(meta_rows)
print("Meta rows shape:", meta_df.shape)
if meta_df.empty:
    raise SystemExit("No person-level ITS results were produced. Check `panel` and min_rows_required.")



# attach person traits
followers = {
    "Abdelfattah Elsisi": 6300000,
    "Alberto Fernández": 2200000,
    "Barack Obama": 129000000,
    "Benjamin Netanyahu": 3500000,
    "Boris Johnson": 4500000,
    "Emmanuel Macron": 10200000,
    "Imran Khan": 21200000,
    "Ivan Duque": 2600000,
    "Jair Bolsonaro": 14100000,
    "Joko Widodo": 21700000,
    "Justin Trudeau": 6600000,
    "Kaguta Museveni": 3600000,
    "King Salman": 10200000,
    "Lopez Obrador": 11000000,
    "Mohammed AlMaktoum": 10900000,
    "Moon Jaein": 2100000,
    "Muhammadu Buhari": 4500000,
    "Nana AkufoAddo": 2700000,
    "Narendra Modi": 108600000,
    "Nicolas Maduro": 4700000,
    "Paul Kagame": 3300000,
    "Queen Rania": 9700000,
    "Sebastian Pinera": 2300000,
    "Tayyip Erdogan": 21300000,
    "Adele Adkins": 25500000,
    "Alecia Beth": 28700000,
    "Alicia Keys": 27700000,
    "Armando Perez": 22800000,
    "Britney Spears": 51600000,
    "Bruno Mars": 40900000,
    "Chris Brown": 30500000,
    "Demi Lovato": 50000000,
    "Drake Graham": 37800000,
    "Harry Styles": 35300000,
    "Jennifer Lopez": 42200000,
    "Justin Bieber": 105700000,
    "Justin Timberlake": 57400000,
    "Kanye West": 32700000,
    "Katy Perry": 101300000,
    "Lady Gaga": 79700000,
    "Liam Payne": 31600000,
    "Lil Wayne": 32900000,
    "Louis Tomlinson": 33500000,
    "Miley Cyrus": 44600000,
    "Niall Horan": 38200000,
    "Nicki Minaj": 27800000,
    "Robyn Rihanna": 105800000,
    "Shakira Ripoll": 51300000,
    "Shawn Mendes": 25200000,
    "Taylor Swift": 91400000,
    "Wiz Khalifa": 34700000,
    "Zayn Malik": 29200000,
    "Akshay Kumar": 46600000,
    "Amitabh Bachchan": 48300000,
    "Deepika Padukone": 25700000,
    "Emma Watson": 25700000,
    "Hrithik Roshan": 31800000,
    "Priyanka Chopra": 26800000,
    "Salman Khan": 45200000,
    "Selena Gomez": 63600000,
    "ShahRukh Khan": 43500000,
    "Andres Iniesta": 24300000,
    "Cristiano Ronaldo": 114200000,
    "LeBron James": 52000000,
    "Mesut Ozil": 25300000,
    "Neymar Junior": 62900000,
    "Ricardo Kaka": 27400000,
    "Sachin Tendulkar": 40500000,
    "Virat Kohli": 67500000,
    "Conan Obrien": 26100000,
    "Jimmy Fallon": 47800000,
    "Kevin Hart": 35200000,
    "Oprah Winfrey": 39400000,
    "Patrick Harris": 23100000,
    "Whindersson Nunes": 27600000,
    "Kendal Jenner": 30600000,
    "Khloe Kardashian": 29200000,
    "Kim Kardashian": 73300000,
    "Kourtney Kardashian": 25200000,
    "Kylie Jenner": 38900000,
    "Bill Gates": 66000000,
    "Elon Musk": 228500000,
    "Eric Yuan": 86800,
    "Jack Dorsey": 6400000,
    "Joe Gebbia": 206100,
    "John Collison": 208500,
    "Ken Fisher": 428300,
    "Marc Benioff": 1100000,
    "Melinda Gates": 2400000,
    "Michael Dell": 796100,
    "Micky Arison": 184800,
    "Mike Bloomberg": 2500000,
    "Mike Brookes": 105100,
    "Orlando Bravo": 44100,
    "Patrick Collison": 572000,
    "Ralph Lauren": 2200000,
    "Ricardo Salinas": 2100000,
    "Samuel Bankman": 996800,
    "Tilman Fertitta": 103800,
    "Tim Sweeney": 316100,
    "Tobi Lutke": 424100,
    "Vinod Khosla": 686500

    # ... include all others or read from a CSV later
}

import unicodedata

def normalize_name(name):
    if not isinstance(name, str):
        return name
    # remove accents and normalize
    name = ''.join(
        c for c in unicodedata.normalize('NFD', name)
        if unicodedata.category(c) != 'Mn'
    )
    # remove spaces and lowercase
    return name.replace(" ", "").lower()

# normalize keys in the followers dictionary
followers_norm = {normalize_name(k): v for k, v in followers.items()}

# normalize names in the meta_df
meta_df["person_norm"] = meta_df["person"].apply(normalize_name)

# map using normalized names
meta_df["followers"] = meta_df["person_norm"].map(followers_norm)
meta_df["log_followers"] = np.log1p(meta_df["followers"])

# Map category: if person_to_cat keys are normalized, adapt similarly
# If person_to_cat keys are normalized names, normalize the meta_df.person similarly
# We'll try both direct map and normalized fallback:
def normalize_person_for_lookup(name):
    return name.replace(" ", "").replace("_", "").lower()

# try direct mapping first
meta_df["category"] = meta_df["person"].map(person_to_cat)

# if many Unknown, try normalized mapping
if meta_df["category"].isna().sum() > 0:
    # build normalized person_to_cat_norm (if not already built)
    person_to_cat_norm = {}
    for k, v in person_to_cat.items():
        # if keys are display names, they may already be normalized; handle both
        person_to_cat_norm[normalize_person_for_lookup(k)] = v
        person_to_cat_norm[normalize_person_for_lookup(k.replace("_Weekly_Behavioral_Summary",""))] = v
    # map using normalized key
    meta_df.loc[meta_df["category"].isna(), "category"] = meta_df.loc[meta_df["category"].isna(), "person"].apply(
        lambda x: person_to_cat_norm.get(normalize_person_for_lookup(x), "Unknown")
    )

# Compute pre-COVID averages per person for the same variables
pre = panel[panel["week"] < week_start(intervention_date)]
pre_means = pre.groupby("person")[[ "tweet_count","avg_sentiment","avg_length",
                                    "prop_tweet","prop_retweet","prop_reply","prop_quote"]].mean()
pre_means = pre_means.add_prefix("pre_")

# Merge pre-period means into meta_df
meta_df = meta_df.merge(pre_means, left_on="person", right_index=True, how="left")

# --- Run meta-regression per outcome variable ---
meta_results = []
for var in outcomes:
    df_var = meta_df[meta_df["variable"] == var].copy()
    if df_var.shape[0] < 10:
        print(f"⚠️ Skipping meta-regression for {var}: only {df_var.shape[0]} persons")
        continue

    # prepare formula: outcome is intervention_coef (level shift) or time_after_coef (slope change)
    # We'll do two meta-regressions per outcome: (A) intervention_coef as dependent, (B) time_after_coef as dependent
    for dep in ["intervention_coef", "time_after_coef"]:
        # Basic predictiors to include (only keep columns that exist and are not all-NaN)
        predictors = ["log_followers", "pre_avg_sentiment", "pre_avg_length", "pre_tweet_count"]
        # rename pre columns mapping expected names
        rename_map = {
            "pre_avg_sentiment": "pre_avg_sentiment",  # but our pre_means column is pre_avg_sentiment? match names below
        }
        # The pre_means columns are named like pre_avg_sentiment? we created pre_ + variable names: pre_avg_sentiment, pre_tweet_count etc
        # ensure they exist:
        available_preds = []
        for p in ["log_followers", "pre_avg_sentiment", "pre_avg_length", "pre_tweet_count"]:
            col = p
            # pre_ columns in meta_df are: pre_avg_sentiment is actually pre_avg_sentiment? check:
            # We created pre_ + original column names -> pre_avg_sentiment exists because original column name was avg_sentiment
            if col in df_var.columns or col in meta_df.columns:
                available_preds.append(col)

        # Clean df_var: drop rows missing dep or predictors
        cols_needed = [dep] + available_preds + ["category"]
        df_clean = df_var[cols_needed].dropna()
        nobs = df_clean.shape[0]
        if nobs < 8:
            print(f" ⚠️ Not enough complete rows for {var} / {dep} ({nobs} rows). Skipping.")
            continue

        # If category has only one level in df_clean, drop it from formula
        cat_levels = df_clean["category"].dropna().unique().tolist()
        include_cat = (len(cat_levels) > 1 and not (len(cat_levels) == 1 and cat_levels[0] in ["", "Unknown"]))
        if include_cat:
            formula = dep + " ~ " + " + ".join(available_preds) + " + C(category)"
        else:
            formula = dep + " ~ " + " + ".join(available_preds)

        try:
            mf = smf.ols(formula, data=df_clean).fit()
            meta_results.append({
                "outcome_var": var,
                "dependent": dep,
                "n_persons": nobs,
                "formula": formula,
                "params": mf.params.to_dict(),
                "bse": mf.bse.to_dict(),
                "pvalues": mf.pvalues.to_dict(),
                "rsquared": mf.rsquared
            })
            print(f" ✅ Meta {var} | {dep}  — n={nobs} formula: {formula}")
        except Exception as e:
            print(f" ❌ Meta regression failed for {var} {dep}: {e}")

# Save meta-level data and results
meta_df.to_csv(output_root / "ITS_meta_person_level_rows.csv", index=False)
import json
with open(output_root / "ITS_meta_regression_results.json", "w", encoding="utf-8") as f:
    json.dump(meta_results, f, indent=2, default=str)

print("✅ Saved meta person-level rows and meta-regression results.")

In [ ]:
# =========================================================
# ✨ FINAL POLISH: Thicker Confidence Lines + Significance Heatmap
# =========================================================
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set_theme(style="whitegrid", context="talk")

light_palette = ["#f4a582", "#92c5de", "#b2df8a"]

def plot_symlog_bar_final(term, df):
    df_plot = df[df["term"] == term].copy()

    vars_order = list(df_plot["variable"].unique())
    df_plot["variable"] = pd.Categorical(df_plot["variable"], categories=vars_order, ordered=True)
    df_plot = df_plot.sort_values(["variable", "category"]).reset_index(drop=True)

    categories = df_plot["category"].unique()
    n_cat = len(categories)
    n_vars = len(vars_order)

    base_positions = np.arange(n_vars)
    offsets = np.linspace(-0.3, 0.3, n_cat)
    bar_width = 0.2

    plt.figure(figsize=(18, 8))
    ax = plt.gca()

    # ---- Draw bars and confidence lines ----
    for i, cat in enumerate(categories):
        subset = df_plot[df_plot["category"] == cat]
        x_positions = base_positions + offsets[i]

        ax.bar(
            x_positions,
            subset["coef"],
            width=bar_width,
            label=cat,
            color=light_palette[i],
            alpha=0.9,
            edgecolor="none",
        )

        # Thicker, darker confidence intervals
        for x, c, se in zip(x_positions, subset["coef"], subset["std_err"]):
            ax.plot(
                [x, x],
                [c - se, c + se],
                color="black",
                lw=2,
                alpha=0.8,
                solid_capstyle="round"
            )

    # ---- Y-axis (Symmetric log) ----
    if term == "intervention":
        ax.set_yscale("symlog", linthresh=0.01)
        yticks = [-1000, -100, -10, -1, -0.1, -0.01,
                  0, 0.01, 0.1, 1, 10, 100, 1000]
    else:
        ax.set_yscale("symlog", linthresh=0.0001)
        yticks = [-1, -0.1, -0.01, -0.001, -0.0001,
                  0, 0.0001, 0.001, 0.01, 0.1, 1]
    ax.set_yticks(yticks)
    ax.set_yticklabels([str(y) for y in yticks])

    # ---- X-axis evenly spaced ----
    ax.set_xticks(base_positions)
    ax.set_xticklabels(vars_order, rotation=0, ha="center")

    # ---- Titles & Labels ----
    ax.set_title(f"{term.title()} Effects (Symmetric Log Y-axis)", fontsize=18)
    ax.set_xlabel("Outcome Variable", fontsize=15)
    ax.set_ylabel("Effect Size (symlog scale)", fontsize=15)

    ax.legend(title="Category", frameon=True, loc="best")

    plt.tight_layout()
    plt.savefig(output_root / f"Fig_Category_{term}_effects_symlog_FINAL.png", dpi=400)
    plt.show()


for term in ["intervention", "time_after"]:
    plot_symlog_bar_final(term, summary_df)


# =========================================================
# (2) Improved Significance Heatmap (compact labels)
# =========================================================
# --- Abbreviations for outcome variables ---
abbr_map = {
    "tweet_count": "Tweets",
    "avg_sentiment": "Sent",
    "avg_length": "Len",
    "prop_tweet": "P_Tw",
    "prop_retweet": "P_RT",
    "prop_reply": "P_Rp",
    "prop_quote": "P_Qt"
}

summary_df["var_short"] = summary_df["variable"].map(abbr_map)
summary_df["cat_term"] = summary_df["category"] + "\n" + summary_df["term"].str.replace("_", " ")

# --- Create significance bin ---
sig_map = (
    summary_df
    .assign(sig_bin=lambda d: np.select(
        [d.p_value < 0.001, d.p_value < 0.01, d.p_value < 0.05],
        [3, 2, 1], 0))
    .pivot_table(index="var_short", columns="cat_term", values="sig_bin", aggfunc="max")
)

# --- Plot heatmap ---
plt.figure(figsize=(18, 10))  # larger figure for clarity

ax = sns.heatmap(
    sig_map,
    cmap="YlGnBu",
    linewidths=0.6,
    linecolor="white",
    annot=True,
    fmt=".0f",
    cbar_kws={
        "label": "Significance Level\n(3: p < 0.001, 2: p < 0.01, 1: p < 0.05, 0: ns ≥ 0.05)"
    }
)

# --- Improve label orientation and clarity ---
plt.title("Significance Levels by Category × Term (Compact, Annotated View)", fontsize=18, pad=20)
plt.xlabel("Category × Term", fontsize=15)
plt.ylabel("Outcome Variable (Abbreviated)", fontsize=15)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)  # horizontal variable labels

# --- Beautify layout ---
plt.tight_layout()
plt.savefig(output_root / "Fig_Significance_Heatmap_Improved_Final.png", dpi=400)
plt.show()
